In [ ]:
import pandas as tian
tian.set_option('display.max_columns', None)
import numpy as lumpnump
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import TensorBoard
import seaborn as sns


import datetime
import gcsfs
from google.cloud import storage as gcs
from google.oauth2 import service_account as gsa

import os
from dotenv import load_dotenv
import certifi

os.environ['SSL_CERT_FILE'] = certifi.where()
top_tracks_stem = os.getenv('TOP_TRACKS_STEM')

In [ ]:
load_dotenv()
# System variables
google_client_id = os.getenv('GOOGLE_CLIENT_ID')
google_client_secret = os.getenv('GOOGLE_CLIENT_SECRET')
google_project_id = os.getenv('GOOGLE_MUSIC_PROJECT')
google_bucket = os.getenv('GOOGLE_PRIMARY_BUCKET')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.getenv('GOOGLE_SERVICE_CREDENTIALS')

# GCS setup details
bucket_file_base = f"gs://{google_bucket}/"
gcp_storage = gcs.Client()
gcp_primary_bucket = gcp_storage.bucket(google_bucket)
gcp_file_system = gcsfs.GCSFileSystem()


## importing bigger files and creating subset I want for neural network

In [ ]:
# toptracks_bpmeta_matrix_csv = f"gs://{google_bucket}/silent_ascent_filestoptracks_bpmeta_matrix"
# toptracks_bpmeta_matrix_df = f"gs://{google_bucket}/silent_ascent_filestoptracks_bpmeta_matrix_df"
# toptracks_bpmeta_matrix_df_csv = '/Users/kevinkirby/Desktop/ml_local_csvs/toptracks_bpmeta_matrix_df'
# toptracks_bpmeta_matrix_df = tian.read_csv(toptracks_bpmeta_matrix_df_csv)

# #matrix
# try:
#   fs = gcsfs.GCSFileSystem(project=google_project_id, token=os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))
#   with fs.open(toptracks_bpmeta_matrix_csv, 'rb') as f:
#     toptracks_bpmeta_matrix = lumpnump.genfromtxt(f, delimiter=',')
#     print('Download complete')
# except Exception as e:
#   print(f"Error: {e}")

# #df
# try:
#   fs = gcsfs.GCSFileSystem(project=google_project_id, token=os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))
#   with fs.open(toptracks_bpmeta_matrix_df, 'rb') as f:
#     toptracks_bpmeta_matrix_df = tian.read_csv(toptracks_bpmeta_matrix_df)
#     print('Download complete')
# except Exception as e:
#   print(f"Error: {e}")

In [ ]:
# toptracks_bpmeta_matrix_df.head()

In [ ]:
# toptracks_bpmeta_matrix_df_columns = toptracks_bpmeta_matrix_df.columns
# toptracks_bpmeta_matrix_df_dict = {name: idx for idx, name in enumerate(toptracks_bpmeta_matrix_df_columns)}
# toptracks_bpmeta_matrix_df_dict

### eda + smaller file

#### eda 

In [ ]:
# toptracks_bpmeta_matrix_df.nunique()

In [ ]:
# # creates some charts to help figure out where the data hangs out
# def metric_breakouts(dataframe, metric, percent_intervals):
#     interval = tian.cut(dataframe[metric], bins=percent_intervals)
#     metric_rows = dataframe.groupby(interval, observed=False).size()
#     intervals_df = metric_rows.reset_index()
#     intervals_df.columns = [f'{metric} range', 'row_count']
#     all_rows = intervals_df['row_count'].sum()
#     intervals_df['% of total'] = (intervals_df['row_count'] / all_rows) * 100
#     intervals_df = intervals_df.sort_values(by='row_count', ascending=False)
#     return intervals_df

# def count_unique_in_intervals(dataframe, interval_field, count_field, interval):
#     dataframe = dataframe.sort_values(by=interval_field, ascending=False)
#     num_intervals = 100 // interval
#     intervals = tian.cut(range(len(dataframe)), bins=num_intervals, labels=[f'{i*interval}-{(i+1)*interval}%' for i in range(num_intervals)])
#     dataframe['interval'] = intervals
#     unique_rows = dataframe.groupby('interval')[count_field].nunique().reset_index(name='unique rows')
    
#     return unique_rows

In [ ]:
# label_points = toptracks_bpmeta_matrix_df.groupby('label_id')['points'].sum()
# total_points = label_points.sum()
# top_20_labels = label_points.nlargest(20)
# top_20_labels_percentage = (top_20_labels / total_points) * 100
# top_labels_df = tian.DataFrame({
#     'points_sum': top_20_labels,
#     'percentage_of_total': top_20_labels_percentage
# })

# print(top_labels_df)

In [ ]:
# label_df = count_unique_in_intervals(toptracks_bpmeta_matrix_df, 'points', 'label_id', 10)
# print(label_df)

In [ ]:
# bpm_interval = range(0, 310, 10)
# bpm_metrics_df = metric_breakouts(toptracks_bpmeta_matrix_df, 'bpm', bpm_interval)
# bpm_metrics_df = bpm_metrics_df.sort_values(by='bpm range', ascending=True)
# print(bpm_metrics_df)

#### histogram and box plots by metric

In [ ]:
histo_box_plot(toptracks_bpmeta_matrix_df, metric_ids)

#### time series histogram by metric and year

In [ ]:
time_series_histo(toptracks_bpmeta_matrix_df, metric_ids, 'release_year')

#### smaller df/matrix based on eda

In [ ]:
# #columns to have in final new df 
# selected_columns = ['isrc_numeric', 'release_year', 'label_percent_interval', 'genre_id_x', 
#                     'duration', 'bpm_group', 'key_id', 'mode', 'valence', 'points']

# # clearing out years that I don't need
# filtered_df = toptracks_bpmeta_matrix_df.loc[
#     (toptracks_bpmeta_matrix_df['release_year'] >= 2019.0) & 
#     (toptracks_bpmeta_matrix_df['release_year'] <= 2024.0)
# ].copy()

# # grouping labels in % intervals based on points 
# filtered_df.loc[:, 'label_percent_interval'] = tian.qcut(filtered_df['points'], 20, labels=False) / 20.0

# #bpm range 
# bpm_range = [0, 60, 100, 110, 120, 130, 140, 150, 300]
# filtered_df.loc[:, 'bpm_group'] = tian.cut(filtered_df['bpm'], bpm_range, right=False)

# # smashing bpm ranges together to create float 
# filtered_df['bpm_group'] = filtered_df['bpm_group'].apply(lambda x: float(f"{int(x.left)}{int(x.right)}"))

# pogi_soar_matrix_df = filtered_df[selected_columns]
# pogi_soar_matrix = pogi_soar_matrix_df.to_numpy()

In [ ]:
# pogi_soar_matrix_df.head()

In [ ]:
# print(pogi_soar_matrix_df.isna().sum())

In [ ]:
# gcp export
# pogi_soar_matrix_df_csv = "pogi_soar_matrix_df.csv"
# pogi_soar_matrix_csv = "pogi_soar_matrix.csv"

# pogi_soar_matrix_df.to_csv(pogi_soar_matrix_df_csv, index=False)
# lumpnump.savetxt(pogi_soar_matrix_csv, pogi_soar_matrix, delimiter=',')

# # Upload to Google Cloud Storage
# def upload_to_gcs(source_file_name, destination_blob_name):
#     blob = gcp_primary_bucket.blob(destination_blob_name)
#     blob.upload_from_filename(source_file_name)

# upload_to_gcs(pogi_soar_matrix_df_csv, f"top_tracks_tables/{pogi_soar_matrix_df_csv}")
# upload_to_gcs(pogi_soar_matrix_csv, f"top_tracks_tables/{pogi_soar_matrix_csv}")

# print("upload complete")

## Import of new and smaller matrix from GCP
This is so I don't have to run all of the above every time. I've saved it to show my work.

In [ ]:
pogi_soar_matrix_csv = f"gs://{google_bucket}/silent_ascent_files/pogi_soar_matrix.csv"

try:
  fs = gcsfs.GCSFileSystem(project=google_project_id, token=os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))
  with fs.open(pogi_soar_matrix_csv, 'rb') as f:
    pogi_soar_matrix = lumpnump.genfromtxt(f, delimiter=',')
    print('Download complete')
except Exception as e:
  print(f"Error: {e}")


# pogi_soar_matrix_csv_local = '/Users/kevinkirby/Desktop/ml_local_csvs/pogi_soar_matrix.csv'
# pogi_soar_matrix = lumpnump.genfromtxt(pogi_soar_matrix_csv_local, delimiter=',')

In [ ]:
#this is a dictionary of indexes from the matrix and a corresponding df not imported and then some values. 
pogi_soar_dict = {
    0: {'name': 'isrc_numeric',
        'sample_value_df': 7218192100041,
        'sample_value_matrix': 7218192100041,
        'unique_count': 75701},
    1: {'name': 'release_year',
        'sample_value_df': 2021,
        'sample_value_matrix': 2021,
        'unique_count': 5},
    2: {'name': 'label_percent_interval',
        'sample_value_df': 0.95,
        'sample_value_matrix': 0.95,
        'unique_count': 20},
    3: {'name': 'genre_id_x',
        'sample_value_df': 12,
        'sample_value_matrix': 12,
        'unique_count': 32},
    4: {'name': 'duration',
        'sample_value_df': 445,
        'sample_value_matrix': 445,
        'unique_count': 693},
    5: {'name': 'bpm_group',
        'sample_value_df': 120130.0,
        'sample_value_matrix': 120120.0,
        'unique_count': 8},
    6: {'name': 'key_id',
        'sample_value_df': 4,
        'sample_value_matrix': 4,
        'unique_count': 34},
    7: {'name': 'mode',
        'sample_value_df': 0,
        'sample_value_matrix': 0,
        'unique_count': 2},
    8: {'name': 'valence',
        'sample_value_df': 0.473,
        'sample_value_matrix': 0.473,
        'unique_count': 1678},
    9: {'name': 'points',
        'sample_value_df': 35920,
        'sample_value_matrix': 35920,
        'unique_count': 7286}
}


In [ ]:
def plot_cat_decision_boundary(ax, X, predict, class_labels=None, legend=False, vector=True, color='g', lw=1):
    pad = 0.5
    x_min, x_max = X[:, 0].min() - pad, X[:, 0].max() + pad
    y_min, y_max = X[:, 1].min() - pad, X[:, 1].max() + pad
    h = max(x_max - x_min, y_max - y_min) / 200
    xx, yy = lumpnump.meshgrid(lumpnump.arange(x_min, x_max, h), lumpnump.arange(y_min, y_max, h))
    points = lumpnump.c_[xx.ravel(), yy.ravel()]
    
    if vector:
        Z = predict(points)
    else:
        Z = lumpnump.zeros((len(points),))
        for i in range(len(points)):
            Z[i] = predict(points[i].reshape(1, 2))
    Z = Z.reshape(xx.shape)
    
    ax.contour(xx, yy, Z, colors=color, linewidths=lw)
    ax.axis('tight')

def plt_mc_data(ax, X, y, classes, class_labels=None, map=plt.cm.Paired, legend=False, size=50, m='o', equal_xy=False):
    for i in range(classes):
        idx = lumpnump.where(y == i)
        col = len(idx[0]) * [i]
        label = class_labels[i] if class_labels else "c{}".format(i)
        ax.scatter(X[idx, 0], X[idx, 1], marker=m, color=map(col), vmin=0, vmax=map.N, s=size, label=label)
    if legend:
        ax.legend()
    if equal_xy:
        ax.axis("equal")


### neural work

In [ ]:
#release year (cat), label percent interval (cat), key id (cat), mode (cat)
two_layer_inputs = [1, 2, 6, 7]
inputs = pogi_soar_matrix[:, two_layer_inputs]
#0,1,2,3 

# genre ID (categorical), bpm_group (cat), valence (continuous value numeric)
two_layer_targets = [3, 5, 8]
targets = pogi_soar_matrix[:, two_layer_targets]
#0,1,2

year_uniques = {sub_array[0] for sub_array in inputs}
label_percent_uniques = {sub_array[1] for sub_array in inputs}
key_uniques = {sub_array[2] for sub_array in inputs}
mode_uniques = {sub_array[3] for sub_array in inputs}

print(f'input years: {year_uniques}')
print(f'input label %: {label_percent_uniques}')
print(f'input keys: {key_uniques}')
print(f'input modes: {mode_uniques}')

genre_uniques = {sub_array[0] for sub_array in targets}
bpm_uniques = {sub_array[1] for sub_array in targets}
valence_uniques = list({sub_array[2] for sub_array in targets})

print(f'target genres: {genre_uniques}')
print(f'target bpms: {bpm_uniques}')
print(f'target valence: {valence_uniques[:10]}')


In [ ]:
def quantile_transform_data(data):
    transformer = QuantileTransformer(output_distribution='normal')
    return transformer.fit_transform(data.reshape(-1, 1)).flatten()

categorical_transformer = OneHotEncoder(sparse_output=False)
numerical_transformer = MinMaxScaler()

valence_index = two_layer_targets.index(8)
valence_data = targets[:, valence_index]
valence_transformed = quantile_transform_data(valence_data)
targets[:, valence_index] = valence_transformed

input_preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', categorical_transformer, [0, 1, 2, 3])  
    ]
)

target_preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', categorical_transformer, [0, 1]),  
        ('numerical', numerical_transformer, [2])        
    ]
)

print("Original matrix shape:", pogi_soar_matrix.shape)

# Transform inputs and outputs and check shapes
inputs_transformed = input_preprocessor.fit_transform(inputs)
print("Transformed inputs shape:", inputs_transformed.shape)

targets_transformed = target_preprocessor.fit_transform(targets)
print("Transformed targets shape:", targets_transformed.shape)

X_train, X_temp, y_train, y_temp = train_test_split(inputs_transformed, targets_transformed, test_size=0.40, random_state=1)
X_cv, X_test, y_cv, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=1)

print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)
print("X_cv.shape:", X_cv.shape)
print("y_cv.shape:", y_cv.shape)
print("X_test.shape:", X_test.shape)
print("y_test.shape:", y_test.shape)

# Extract each input feature from the transformed inputs
X_train_year = X_train[:, 0].reshape(-1, 1)
X_train_label = X_train[:, 1].reshape(-1, 1)
X_train_key = X_train[:, 2].reshape(-1, 1)
X_train_mode = X_train[:, 3].reshape(-1, 1)

print("X_train_year:", X_train_year.shape)
print("X_train_label:", X_train_label.shape)
print("X_train_key:", X_train_key.shape)
print("X_train_mode:", X_train_mode.shape)

X_cv_year = X_cv[:, 0].reshape(-1, 1)
X_cv_label = X_cv[:, 1].reshape(-1, 1)
X_cv_key = X_cv[:, 2].reshape(-1, 1)
X_cv_mode = X_cv[:, 3].reshape(-1, 1)

print("X_cv_year:", X_cv_year.shape)
print("X_cv_label:", X_cv_label.shape)
print("X_cv_key:", X_cv_key.shape)
print("X_cv_mode:", X_cv_mode.shape)

X_test_year = X_test[:, 0].reshape(-1, 1)
X_test_label = X_test[:, 1].reshape(-1, 1)
X_test_key = X_test[:, 2].reshape(-1, 1)
X_test_mode = X_test[:, 3].reshape(-1, 1)

print("X_test_year:", X_test_year.shape)
print("X_test_label:", X_test_label.shape)
print("X_test_key:", X_test_key.shape)
print("X_test_mode:", X_test_mode.shape)

y_train_genre = y_train[:, 0]
y_train_bpm = y_train[:, 1]
y_train_valence = y_train[:, 2]

print("y_train_genre:", y_train_genre.shape)
print("y_train_bpm:", y_train_bpm.shape)
print("y_train_valence:", y_train_valence.shape)

y_cv_genre = y_cv[:, 0]
y_cv_bpm = y_cv[:, 1]
y_cv_valence = y_cv[:, 2]

print("y_cv_genre:", y_cv_genre.shape)
print("y_cv_bpm:", y_cv_bpm.shape)
print("y_cv_valence:", y_cv_valence.shape)

y_test_genre = y_test[:, 0]
y_test_bpm = y_test[:, 1]
y_test_valence = y_test[:, 2]

print("y_test_genre:", y_test_genre.shape)
print("y_test_bpm:", y_test_bpm.shape)
print("y_test_valence:", y_test_valence.shape)

In [ ]:
def pogi_prep():
    soigneur = Sequential([
        Input(shape=(1,)),
        Dense(25, activation='relu', kernel_regularizer=l2(0.01)),
        Dense(17, activation='relu', kernel_regularizer=l2(0.01)),
    ])
    return soigneur

label_input = Input(shape=(1,), name='label_percent_interval')
key_input = Input(shape=(1,), name='key')
mode_input = Input(shape=(1,), name='mode')
release_year_input = Input(shape=(1,), name='release_year')

In [ ]:
def pogi_slay():
    label_features = pogi_prep()(label_input)
    key_features = pogi_prep()(key_input)
    mode_features = pogi_prep()(mode_input)
    release_year_features = pogi_prep()(release_year_input)

    false_flat = Concatenate()([label_features, key_features, mode_features, release_year_features])

    dolly_hidden = Dense(13, activation='relu', kernel_regularizer=l2(0.1))(false_flat)
    pogi_hidden = Dense(10, activation='relu', kernel_regularizer=l2(0.1))(dolly_hidden)

    genre_slay = Dense(32, activation='softmax', name='genre_id')(pogi_hidden)
    bpm_4tofloor = Dense(8, activation='softmax', name='bpm_group')(pogi_hidden)
    valence_smile = Dense(1, activation='sigmoid', name='valence')(pogi_hidden)
    
    pogi_wheels = Model(inputs=[label_input, key_input, mode_input, release_year_input], 
                        outputs=[genre_slay, bpm_4tofloor, valence_smile])

    return pogi_wheels

In [ ]:
ventoux_summit = pogi_slay()
ventoux_summit.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'genre_id': SparseCategoricalCrossentropy(),
        'bpm_group': SparseCategoricalCrossentropy(),
        'valence': MeanSquaredError()
    },
    metrics={
        'genre_id': ['accuracy', Precision(), Recall()],
        'bpm_group': ['accuracy', Precision(), Recall()],
        # 'valence': ['mae','mean_squared_error', 'RootMeanSquaredError'],
        'valence': 'mean_squared_error',
    }
)

### model run

In [ ]:
pogi_stem = '/Users/kevinkirby/Desktop/project_silent_ascent/pogi_logs'
pogi_logger = f'{pogi_stem}/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}-ventoux_summit'
# pogi_stem = f'{bucket_file_base}/silent_ascent_files/pogi_logs'
# pogi_logger = f'{pogi_stem}/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}-ventoux_summit'


pogi_monitoring = TensorBoard(log_dir=pogi_logger, histogram_freq=1)

# fitting
pogi_soar = ventoux_summit.fit(
    [X_train_year, X_train_label, X_train_key, X_train_mode], 
    {'genre_id': y_train_genre, 'bpm_group': y_train_bpm, 'valence': y_train_valence}, 
    epochs=10, 
    batch_size=100,
    validation_data=(
        [X_cv_year, X_cv_label, X_cv_key, X_cv_mode], 
        {'genre_id': y_cv_genre, 'bpm_group': y_cv_bpm, 'valence': y_cv_valence}
    ),
    callbacks=[pogi_monitoring]
)

### plot predict

In [ ]:
predictions = ventoux_summit.predict([X_test_year, X_test_label, X_test_key, X_test_mode])
print(predictions)

In [ ]:
ventoux_evaluation = ventoux_summit.evaluate([X_test_year, X_test_label, X_test_key, X_test_mode], y_test)


In [ ]:
# Extract predictions
predictions_genre = predictions[0]
print(f'genre{predictions_genre}')
predictions_bpm = predictions[1]
print(f'bpm{predictions_bpm}')
predictions_valence = predictions[2]


In [ ]:
# Extract predictions
predictions_genre = predictions[0]
predictions_bpm = predictions[1]
predictions_valence = predictions[2]

# Convert probabilities to predicted classes
predicted_genre_classes = lumpnump.argmax(predictions_genre, axis=1)
predicted_bpm_classes = lumpnump.argmax(predictions_bpm, axis=1)

plt.figure(figsize=(15, 10))

# Use seaborn heatmap for better visualization
sns.heatmap(predictions_genre, cmap='viridis', xticklabels=[f'Genre {i+1}' for i in range(predictions_genre.shape[1])], yticklabels=False)

plt.xlabel('Genre Classes')
plt.ylabel('Samples')
plt.title('Predicted Probabilities for Each Genre Class')
plt.show()

In [ ]:
def plot_loss_tf(history):
    fig,ax = plt.subplots(1,1, figsize = (4,3))
    widgvis(fig)
    ax.plot(history.history['loss'], label='loss')
    ax.set_ylim([0, 2])
    ax.set_xlabel('Epoch')
    ax.set_ylabel('loss (cost)')
    ax.legend()
    ax.grid(True)
    plt.show()

In [ ]:
# # terminal command for tensorboard
# !tensorboard --logdir=/Users/kevinkirby/Desktop/project_silent_ascent/pogi_logs
# tensorboard --logdir=gs://love-uwsthoughts/silent_ascent_files/pogi_logs